# Lab 02: Credit Ratings

---
author: Your Name Here
date: February 5, 2024
embed-resources: true
---

## Introduction

## Methods

In [395]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.experimental import enable_iterative_imputer
from joblib import dump

### Data

In [354]:
credit_train = pd.read_csv("https://cs307.org/lab-02/data/credit-train.csv")
# create X and y for train
X_train = credit_train.drop("Rating", axis=1)  # delete the rating column
y_train = credit_train["Rating"]

The Credit Ratings dataset originates from the textbook "An Introduction to Statistical Learning." The modified Credit Ratings dataset for CS 307 Lab 02 includes data for predicting individual consumer credit scores, featuring attributes such as income, age, education, gender, student status, marital status, and ethnicity.

In [355]:
credit_train

,Rating,Income,Age,Education,Gender,Student,Married,Ethnicity
0,448.0,49.570,28.0,9.0,Female,No,Yes,Asian
1,411.0,26.813,55.0,16.0,Female,No,No,Caucasian
2,181.0,30.406,79.0,14.0,Male,No,Yes,African American
3,169.0,27.349,51.0,16.0,Female,No,Yes,African American
4,292.0,12.068,44.0,18.0,Female,No,Yes,Asian
...,...,...,...,...,...,...,...,...
295,369.0,40.442,81.0,8.0,Female,No,No,African American
296,272.0,NaN,69.0,8.0,Male,No,Yes,Caucasian
297,251.0,14.132,75.0,17.0,Male,No,No,Caucasian
298,747.0,135.118,81.0,15.0,Female,No,Yes,Asian


In [356]:
# create validation train and validation 
X_vtrain, X_validation, y_vtrain, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#### Summary Statistics

In [357]:
# Rating (Target) Statistics
mean_rating = y_train.mean()
std_rating = y_train.std()
print("Rating Mean: ", mean_rating, "Rating Standard Deviation: ", std_rating)

Rating Mean:  357.51666666666665 Rating Standard Deviation:  157.09976929943386


In [358]:
# Rating Statistics Based on Student Status
credit_notstudent = credit_train[credit_train["Student"] == "No"]
credit_student = credit_train[credit_train["Student"] == "Yes"]
mean_notstudent = credit_notstudent["Rating"].mean()
std_notstudent = credit_notstudent["Rating"].std()
mean_student = credit_student["Rating"].mean()
std_student = credit_student["Rating"].std()
print("Count for not student: ", len(credit_notstudent), "\nRating Mean for Not Student: ",
      mean_notstudent, "\nRating Standard Deviation for Not Student: ", std_notstudent)
print("\nCount for student: ", len(credit_student), "\nRating Mean for Student: ", mean_student, "\nRating Standard Deviation for Student: ", std_student)

Count for not student:  266 
Rating Mean for Not Student:  359.24436090225566 
Rating Standard Deviation for Not Student:  158.38314085692082

Count for student:  30 
Rating Mean for Student:  344.1 
Rating Standard Deviation for Student:  146.64626686761207


In [359]:
# Rating Statistics Based on Marriage Status
credit_single = credit_train[credit_train["Married"] == "No"]
credit_married = credit_train[credit_train["Married"] == "Yes"]
mean_single = credit_single["Rating"].mean()
std_single = credit_single["Rating"].std()
mean_married = credit_married["Rating"].mean()
std_married = credit_married["Rating"].std()
print("Count for single: ", len(credit_single), "\nRating Mean for Single: ", mean_single, "\nRating Standard Deviation for Single: ", std_single)
print("\nCount for married: ", len(credit_married), "\nRating Mean for Married: ", mean_married, "\nRating Standard Deviation for Married: ", std_married)

Count for single:  118 
Rating Mean for Single:  359.3898305084746 
Rating Standard Deviation for Single:  154.6773887695038

Count for married:  178 
Rating Mean for Married:  358.2752808988764 
Rating Standard Deviation for Married:  159.8516528251666


In [360]:
# Correlation with Age and Income
print("age correlation: ", credit_train["Rating"].corr(credit_train["Age"]))
print("income correlation: ", credit_train["Rating"].corr(credit_train["Income"]))

age correlation:  0.14869459512179506
income correlation:  0.7955279488315458


In [361]:
# Missing Proportion
missing_proportion = credit_train.isnull().mean()
print(missing_proportion)

Rating       0.000000
Income       0.146667
Age          0.096667
Education    0.070000
Gender       0.016667
Student      0.013333
Married      0.013333
Ethnicity    0.063333
dtype: float64


In [362]:
credit_data_dummies = pd.get_dummies(credit_train)
credit_data_dummies

,Rating,Income,Age,Education,Gender_ Male,Gender_Female,Student_No,Student_Yes,Married_No,Married_Yes,Ethnicity_African American,Ethnicity_Asian,Ethnicity_Caucasian
0,448.0,49.570,28.0,9.0,False,True,True,False,False,True,False,True,False
1,411.0,26.813,55.0,16.0,False,True,True,False,True,False,False,False,True
2,181.0,30.406,79.0,14.0,True,False,True,False,False,True,True,False,False
3,169.0,27.349,51.0,16.0,False,True,True,False,False,True,True,False,False
4,292.0,12.068,44.0,18.0,False,True,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,369.0,40.442,81.0,8.0,False,True,True,False,True,False,True,False,False
296,272.0,NaN,69.0,8.0,True,False,True,False,False,True,False,False,True
297,251.0,14.132,75.0,17.0,True,False,True,False,True,False,False,False,True
298,747.0,135.118,81.0,15.0,False,True,True,False,False,True,False,True,False


In [363]:
credit_data_dummies.corr()

,Rating,Income,Age,Education,Gender_ Male,Gender_Female,Student_No,Student_Yes,Married_No,Married_Yes,Ethnicity_African American,Ethnicity_Asian,Ethnicity_Caucasian
Rating,1.000000,0.795528,0.148695,0.018278,-0.012343,0.023150,0.030812,-0.028515,0.009617,0.005843,0.086974,-0.020867,-0.050253
Income,0.795528,1.000000,0.171327,-0.069554,0.022371,-0.001303,0.048192,-0.063218,0.012532,0.002551,0.101959,0.014985,-0.087834
Age,0.148695,0.171327,1.000000,0.026958,-0.047580,0.058471,0.072061,-0.083976,0.014764,-0.019441,0.035452,-0.115021,0.081064
Education,0.018278,-0.069554,0.026958,1.000000,0.052803,-0.076088,-0.071900,0.080712,-0.045129,0.022508,-0.025109,0.002250,0.019986
Gender_ Male,-0.012343,0.022371,-0.047580,0.052803,1.000000,-0.967152,0.062784,-0.069013,0.007383,-0.008974,0.001579,-0.012085,-0.018637
Gender_Female,0.023150,-0.001303,0.058471,-0.076088,-0.967152,1.000000,-0.053577,0.057798,-0.021480,0.022085,0.001051,0.000206,0.022403
Student_No,0.030812,0.048192,0.072061,-0.071900,0.062784,-0.053577,1.000000,-0.932352,-0.121120,0.153558,-0.001657,-0.039355,0.074448
Student_Yes,-0.028515,-0.063218,-0.083976,0.080712,-0.069013,0.057798,-0.932352,1.000000,0.095533,-0.131195,-0.026270,0.067017,-0.082533
Married_No,0.009617,0.012532,0.014764,-0.045129,0.007383,-0.021480,-0.121120,0.095533,1.000000,-0.972603,0.088197,-0.112497,0.028951
Married_Yes,0.005843,0.002551,-0.019441,0.022508,-0.008974,0.022085,0.153558,-0.131195,-0.972603,1.000000,-0.104822,0.095922,-0.003905


In [364]:
# visualizations


### Models

In [365]:
# find data types of X_train(7 features in total)
X_vtrain.dtypes

Income       float64
Age          float64
Education    float64
Gender        object
Student       object
Married       object
Ethnicity     object
dtype: object

In [366]:
# split the data into numerical and categorical
numeric_features=X_vtrain.select_dtypes(include=["float64"]).columns
categorical_features=X_vtrain.select_dtypes(include=["object"]).columns

In [367]:
# define how to handle missing data and scale features for numeric and categorical features
numeric_transformer = Pipeline(
    steps=[
        ("Iterative Imputer", IterativeImputer(max_iter=10, random_state=0)),
        ("Standardization", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    steps=[
        ("Modal Imputer", SimpleImputer(strategy="most_frequent")),
        ("One-Hot Encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

In [368]:
# create general preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("Numeric Transformer", numeric_transformer, numeric_features),
        ("Categorical Transformer", categorical_transformer, categorical_features),
    ],
    remainder="drop",
)

In [399]:
# Create the pipeline
pipeline = Pipeline(steps=[("Preprocessor", preprocessor), ("Classifier", KNeighborsClassifier())])

In [400]:
# define the parameter grid for grid search
param_grid_100 = {
    "Classifier__n_neighbors": range(1,10),
    "Classifier__metric": ["euclidean", "manhattan"],
}


In [401]:
# setup grid search with cross-validation
grid_search_100 = GridSearchCV(pipeline, param_grid_100, cv=2)  
#grid_search_200 = GridSearchCV(pipeline, param_grid_200, cv=3)

In [402]:
# fit models
grid_search_100.fit(X_vtrain, y_vtrain)
#grid_search_200.fit(X_vtrain, y_vtrain)

c:\Users\86188\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('Preprocessor',
                                        ColumnTransformer(transformers=[('Numeric '
                                                                         'Transformer',
                                                                         Pipeline(steps=[('Iterative '
                                                                                          'Imputer',
                                                                                          IterativeImputer(random_state=0)),
                                                                                         ('Standardization',
                                                                                          StandardScaler())]),
                                                                         Index(['Income', 'Age', 'Education'], dtype='object')),
                                                                        ('Categorical '
                                                                         'Transformer',
                                                                         Pipeline(steps=[('Modal '
                                                                                          'Imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('One-Hot '
                                                                                          'Encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['Gender', 'Student', 'Married', 'Ethnicity'], dtype='object'))])),
                                       ('Classifier', KNeighborsClassifier())]),
             param_grid={'Classifier__metric': ['euclidean', 'manhattan'],
                         'Classifier__n_neighbors': range(1, 10)})

In [403]:
vtrain_pred_100=grid_search_100.predict(X_validation)
#vtrain_pred_200=grid_search_200.predict(X_vtest)

In [404]:
# def RMSE():
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [405]:
print("RMSE for 100: ", RMSE(y_validation, vtrain_pred_100))

RMSE for 100:  146.01717935457685


In [406]:
# save models
# dump(grid_search, "credit-ratings.joblib")

In [407]:
from sklearn.neighbors import KNeighborsRegressor
knn010=KNeighborsRegressor(n_neighbors=10)

## Results

In [408]:
# report model metrics

## Discussion

### Conclusion